In [1]:
import numpy as np
from tqdm import tqdm
import os
import json
from torch.utils.data import Dataset, Sampler
import torch.utils.data as data
from Network import Network
from Evaluator import Evaluator

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torchvision
from torchvision import datasets,models,transforms
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import math
import os
import copy
import time
import sqlite3 as sql
import cv2
from tqdm import tqdm
from sam_implem import *
from operator import itemgetter

/home/mdelabrassinne/anaconda3/envs/envDeepLearning/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mdelabrassinne/anaconda3/envs/envDeepLearning/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/mdelabrassinne/anaconda3/envs/envDeepLearning/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")


In [2]:
!pip install wandb -qU

In [2]:
import wandb

### Get features, labels and timecodes

In [3]:
features = {}
json_paths = {}
labels = {}
timeLabel = {}

features['train'] = []
features['validation'] = []
features['test'] = []

labels['train'] = []
labels['validation'] = []
labels['test'] = []

timeLabel['train'] = []
timeLabel['validation'] = []
timeLabel['test'] = []

json_paths['train'] = []
json_paths['validation'] = []
json_paths['test'] = []


path = '/scratch/users/mdelabrassinne/Database/'

nbMatch = 0

val = np.zeros((500))
val[450:475] = 1
val[475:500] = 2
val = val.astype(int)
np.random.seed(42)
np.random.shuffle(val)
types = ["train", "validation", "test"]
current = types[val[nbMatch]]



for root, dirs, files in os.walk(path):
    #print(files)
    
    for file in files:    
        if file.endswith("Labels-v2.json"):
            json_paths[current].append(os.path.join(root, file))
            json_paths[current].append(os.path.join(root, file))
            p = os.path.dirname(os.path.join(root, file))
            
            dire = os.path.basename(p).replace('_', ' ')
            p = os.path.join(os.path.dirname(p), dire)
            
            for r, d, fs in os.walk(p):
                for f in fs:
                    if f.endswith("baidu_soccer_embeddings.npy"):
                        p_current = os.path.join(r, f)
                        features[current].append(p_current)
                        #print(p_current)

            
            jsonPath =(os.path.join(root, file))
            #print(jsonPath)
            
            #opening of the json
            
            f = open(jsonPath)
            d = json.load(f)
            #print(d)
            
            #print(jsonPath)
            
            try:
                #print('test')
                lab = [d['annotations'][i]['label'] for i in range(len(d['annotations'])) if d['annotations'][i]['gameTime'][0] ==  '1']         # 1fps, don't know if keep 
                time = [60 * int(d['annotations'][i]['gameTime'].split(':')[0][-2:]) + int(d['annotations'][i]['gameTime'].split(':')[1][:2]) for i in range(len(d['annotations'])) if d['annotations'][i]['gameTime'][0] ==  '1']
                #print([d['annotations'][i]['gameTime'] for i in range(len(d['annotations']))])
                
                #print(time)
                
                labels[current].append(lab)
                timeLabel[current].append(time)
                
                lab = [d['annotations'][i]['label'] for i in range(len(d['annotations'])) if d['annotations'][i]['gameTime'][0] ==  '2']         # 1fps, don't know if keep 
                time = [60 * int(d['annotations'][i]['gameTime'].split(':')[0][-2:]) + int(d['annotations'][i]['gameTime'].split(':')[1][:2]) for i in range(len(d['annotations'])) if d['annotations'][i]['gameTime'][0] ==  '2']
                
                #print(time)
                labels[current].append(lab)
                timeLabel[current].append(time)
                
            except:
                #print(d)
                lab = [d['predictions'][i]['label'] for i in range(len(d['predictions'])) if d['predictions'][i]['gameTime'][0] == '1']         # 1fps, don't know if keep 
                time = [60 * int(d['predictions'][i]['gameTime'].split(':')[0][-2:]) + int(d['predictions'][i]['gameTime'].split(':')[1][:2]) for i in range(len(d['predictions'])) if d['predictions'][i]['gameTime'][0] == '1']
                
                labels[current].append(lab)
                timeLabel[current].append(time)
                
                lab = [d['predictions'][i]['label'] for i in range(len(d['predictions'])) if d['predictions'][i]['gameTime'][0] == '2']         # 1fps, don't know if keep 
                time = [60 * int(d['predictions'][i]['gameTime'].split(':')[0][-2:]) + int(d['predictions'][i]['gameTime'].split(':')[1][:2]) for i in range(len(d['predictions'])) if d['predictions'][i]['gameTime'][0] == '2']
                
                labels[current].append(lab)
                timeLabel[current].append(time)
                
           
            nbMatch += 1     #one more match will be seen
            
            if (nbMatch >= 500):
                current = "train"
            else:
                current = types[val[nbMatch]]
            if nbMatch % 20 == 0:
                print(nbMatch)
            
            
            

            
data_l = list(zip(features['train'], labels['train'], timeLabel['train'], json_paths['train']))
data_val = list(zip(features['validation'], labels['validation'], timeLabel['validation'], json_paths['validation']))
data_test = list(zip(features['test'], labels['test'], timeLabel['test'], json_paths['test']))

20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500


In [4]:
print(len(data_l))
print(len(data_val))
print(len(data_test))

900
50
50


In [5]:
dataset = data_val # change to data_test for test_set
features1 = []
features2 = []
json_sols = []
for i in range(len(dataset)//2):
    features1.append(dataset[2 * i][0])
    features2.append(dataset[2 * i + 1][0])
    json_sols.append(dataset[2 * i][3])
    if dataset[2 * i][3] != dataset[2 * i + 1][3]:
        print(dataset[2 * i][3])
        print(dataset[2 * i + 1][3])

### Function for the transformation of the ground truths

In [6]:
nb = np.array([2566, 31810, 18918, 11674, 7896, 10521, 5820, 4836, 5256, 2200, 2047, 2839, 55, 2098, 1703, 173, 46])

In [7]:
cL = ['Kick-off', 'Ball out of play', 'Throw-in', 'Foul', 'Clearance', 'Indirect free-kick', 'Shots on target', 'Corner', 'Shots off target', 'Direct free-kick', 'Yellow card', 'Substitution', 'Red card', 'Offside', 'Goal', 'Penalty', 'Yellow->red card']

def getTrueCoef(label, timeCodes, T, nbClasses, r_c = 3, f = 1, r_d = 6):
    C = []      #confidence as a 2D np array instead of a list of tupes

    ## Init
    res_conf = np.zeros((T, nbClasses))         # initialization of the output C
    res_disp = np.zeros((T, nbClasses))         # initialization of the output D
    domain_disp = np.zeros((T, nbClasses))      # initialization of the domain for D ==> the domain has 1 when the value has to be computed for the loss on the displacement and 0 otherwise
    
    dictClasses = {}    # dictionnary containing the list wiht the timecodes of the actions for a given class

    ## Init of the dictionnary
    for c in range(nbClasses):
        dictClasses[c] = []

    ## Focus on the confidence part
    for i in range(len(timeCodes)):        #loop over obtained labels
        t = timeCodes[i]
        c = label[i]
        
        if(c in cL):                           # focus on wanted classes
            c = cL.index(label[i])

            # get the interval on which C contains 1 for a given class
            tmin = max(0, t - r_c * f)        # cannot be lower than 0
            tmax = min(t + r_c * f, T-1)      # cannot be larger than T-1
            inds = np.arange(tmin, tmax+1)    # select the timecodes between tmin and tmax
            res_conf[inds, c] = 1             # put 1 in C at the good position
            
            ## Update the dictionnary
            dictClasses[c].append(t)
            
        ## test for bad cases (don't occur)
        if(t > T):
            print('\n\naie => t = {} -- T = {}\n\n'.format(t, T))
            
        if(i >= 2700):
            print("possible issue ")
            
            
    ## Focus on the displacement part
    for c in range(nbClasses):
        ind = np.array(dictClasses[c])       # find the list of timecodes for a given action
        
        if(len(ind) > 0):
            for t in range(T):           # go through all frames

                ## Find the frame containing an occurence which is the closest (and get the corresponding index)
                try:
                    closestInd = np.argmin(np.abs(ind - t))         #take the one that is the most on the left if draw
                except:
                    print(ind)
                    print(t)
                    print(len(ind))

                ## Check if in the domain
                if(np.abs(t - ind[closestInd]) <= r_d * f):
                    res_disp[t, c] = t - ind[closestInd]      # put the displacement in the matrix
                    domain_disp[t, c] = 1                     # keep the value in the domain
            
        
    return res_conf, res_disp, domain_disp

test_C, test_D, dom = getTrueCoef(data_l[0][1],data_l[0][2], np.load(data_l[0][0]).shape[0], len(cL) )

### Dataset, dataloader

In [8]:
cL = ['Kick-off', 'Ball out of play', 'Throw-in', 'Foul', 'Clearance', 'Indirect free-kick', 'Shots on target', 'Corner', 'Shots off target', 'Direct free-kick', 'Yellow card', 'Substitution', 'Red card', 'Offside', 'Goal', 'Penalty', 'Yellow->red card']

class BaiduDataset(Dataset):    #modifiable
    """
    Class to prepare the data for a neural network in pyTorch. The database is composed of 3 tables:
    - Video: to save the video information
    - Sequence: to save the sequence information labelled with the soccer actions and record the sound of the sequence
    - Image: to save the image information of the sequence
    """
    def __init__(self, data_list, transform=None):
        self.data_list = data_list
        self.transform = transform
        self.seq_length = 64           # number of frames in each sequence
        self.nb_per_item = 10           # number of sequences per item selected
        self.border = 12
        
        ## dynamic dictionnary for classes
        d = {}
        for c in cL:
            d[c] = 1
        self.dictClasses = d
        
        
    def updateDict(self, labels, timecodes, startTime,endTime):
        
        indices = np.where(np.logical_and(timecodes>=startTime, timecodes<=endTime))[0]             #indices verifying the condition
        seenClasses = labels[indices]                                                         #classes occuring in the sequence   
        for classSeen in seenClasses:
            self.dictClasses[classSeen] += 1                                                  #adding occurence to the vector
            
        
    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, idx):
        feat_path = self.data_list[idx][0]                 # path towards Baidu features
        label_list = np.array(self.data_list[idx][1])      # list of labels
        time_list = np.array(self.data_list[idx][2])       # list of timecodes

        features = np.load(feat_path)

        features_out_list = []
        coef_out_c = []
        coef_out_d = []
        dom_out_list = []
        inf_list = []

        for _ in range(self.nb_per_item):
            presentLabels = np.unique(label_list)                                        # only focus on current classes
                        
            weights = np.array(list(itemgetter(*presentLabels)(self.dictClasses)))       # get weights form the dictionnary for those classes
                        
            weights = weights / sum(weights)                                                      # normalize the associated probability vector
            complWeight = 1-weights                                                      # complementary of the probabilities  (more importance for unlikely events)
            complWeight /= sum(complWeight)                                              # normalize the new probabilities   
                        
            selectedClass = presentLabels[np.argmax(np.random.multinomial(1, complWeight, size=1))]         # select the class according to the weights
            
            events = np.where(label_list == selectedClass)[0]                            # consider the indices of the selected class
                        
            selectedEvent = events[np.random.randint(0, len(events))]                    # select the event of the class among the different possibilities (and only consider the index)
                        
            selectedTimestep = time_list[selectedEvent]
                                    
            
            selectedStart = np.random.randint(max(0, selectedTimestep - self.seq_length), min(features.shape[0] - self.seq_length+1, max(1, selectedTimestep)))
                        
            self.updateDict(label_list, time_list, selectedStart, selectedStart+self.seq_length)
                        
            ind = np.array([True if (i >= selectedStart and i < selectedStart + self.seq_length) else False for i in range(features.shape[0])])

            coef_c, coef_d, dom = getTrueCoef(label_list, time_list, features.shape[0], len(cL) )

            features_out = torch.from_numpy(features[ind,:]).unsqueeze(0)
            coef_out_c.append(torch.from_numpy(coef_c[ind, :]).unsqueeze(0))
            coef_out_d.append(torch.from_numpy(coef_d[ind, :]).unsqueeze(0))
            dom_out_list.append(torch.from_numpy(dom[ind, :]).unsqueeze(0))
            features_out_list.append(features_out)

        return (torch.cat(features_out_list, dim=0), torch.cat(coef_out_c, dim=0), torch.cat(coef_out_d, dim=0), torch.cat(dom_out_list, dim=0), idx) 

### Model

In [9]:
MLP_KERNEL_SIZE = 1
MLP_FILTERS_1 = 256
MLP_FILTERS_2 = 64
CONTRACTION_RATIO = 2
PADDING_METHOD = 'same'
UPSAMPLE_MODE = 'nearest'
N_FEATURE_EXTRACTED = 8576
N_CLASSES = 17
SEQUENCE_SIZE = 64
BATCH_SIZE = 20
BATCH_PER_EPOCH = 1000
FPS = 1

In [10]:
class MLP2(nn.Module):
    def __init__(self,n_features):
        super(MLP2,self).__init__()
        self.conv1 = nn.Conv1d(n_features,MLP_FILTERS_1,MLP_KERNEL_SIZE)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(MLP_FILTERS_1,MLP_FILTERS_2,MLP_KERNEL_SIZE)

    def forward(self,x):
        out = self.conv1(x)
        out = self.relu(out)
        out = self.conv2(out)
        return out


In [11]:
class BottleneckResBlockContract(nn.Module):
    # Takes as input a feature matrix of size:
    #  _____
    # |     |
    # |     |
    # |     |
    # |     | T
    # |     |
    # |_____|
    #    P
    # Performs a bottleneck Resnetblock:
    # BN -> ReLU -> conv(1,P) ->BN ->ReLU -> conv(3,P) -> BN -> ReLU -> conv(1,2P) -> + -> output (of size T,2P)
    # |                                                                               ^
    # L>    ->  ->  ->  ->  ->  ->  ->  -> conv(1,2P) ->  ->  ->  ->    ->  ->  ->  ->|  
    def __init__(self,in_features):
        super(BottleneckResBlockContract,self).__init__()
        self.bn1 = nn.BatchNorm1d(in_features)
        self.relu = nn.ReLU()
        self.conv1 = nn.Conv1d(in_features,in_features,1)
        self.bn2 = nn.BatchNorm1d(in_features)
        self.conv2 = nn.Conv1d(in_features,in_features,3,padding=PADDING_METHOD)
        self.bn3 = nn.BatchNorm1d(in_features)
        self.conv3 = nn.Conv1d(in_features,in_features*CONTRACTION_RATIO,1)
        self.conv_skip = nn.Conv1d(in_features,in_features*CONTRACTION_RATIO,1)

    def forward(self,x):
        y = self.bn1(x)
        y = self.relu(y)
        y = self.conv1(y)
        y = self.bn2(y)
        y = self.relu(y)
        y = self.conv2(y)
        y = self.bn3(y)
        y = self.relu(y)
        y = self.conv3(y)
        z = self.conv_skip(x)
        return y+z
    
class BottleneckResBlockExpand(nn.Module):
    # Takes as input a feature matrix of size:
    #  _____
    # |     |
    # |     |
    # |     |
    # |     | T
    # |     |
    # |_____|
    #    P
    # Performs a bottleneck Resnetblock:
    # BN -> ReLU -> conv(1,P/2) ->BN ->ReLU -> conv(3,P/2) -> BN -> ReLU -> conv(1,P/2) -> + -> output (of size T,P/2)
    # |                                                                               ^
    # L>    ->  ->  ->  ->  ->  ->  ->  -> conv(1,P/2) ->  ->  ->  ->    ->  ->  ->  ->|  
    def __init__(self,in_features):
        in_features = int(in_features)
        super(BottleneckResBlockExpand,self).__init__()
        self.bn1 = nn.BatchNorm1d(int(in_features))
        self.relu = nn.ReLU()
        self.conv1 = nn.Conv1d(in_features,int(in_features/CONTRACTION_RATIO),1)
        self.bn2 = nn.BatchNorm1d(int(in_features/CONTRACTION_RATIO))
        self.conv2 = nn.Conv1d(int(in_features/CONTRACTION_RATIO),int(in_features/CONTRACTION_RATIO),3,padding=PADDING_METHOD)
        self.bn3 = nn.BatchNorm1d(int(in_features/CONTRACTION_RATIO))
        self.conv3 = nn.Conv1d(int(in_features/CONTRACTION_RATIO),int(in_features/CONTRACTION_RATIO),1)
        self.conv_skip = nn.Conv1d(in_features,int(in_features/CONTRACTION_RATIO),1)

    def forward(self,x):
        y = self.bn1(x)
        y = self.relu(y)
        y = self.conv1(y)
        y = self.bn2(y)
        y = self.relu(y)
        y = self.conv2(y)
        y = self.bn3(y)
        y = self.relu(y)
        y = self.conv3(y)
        z = self.conv_skip(x)
        return y+z

class UpsamplingBlock(nn.Module):
# Takes as input a feature matrix of size (with ncol: nFeatures and nrow: nTimePoints) :
    #  _____
    # |     |
    # |     |
    # |     |
    # |     | T
    # |     |
    # |_____|
    #    P
# Upsample uses nearest neighbor to output a 2TxP matrix
# a Conv(2,P/2) is then applied with padding = same
    
    def __init__(self,in_feature):
        super(UpsamplingBlock,self).__init__()
        in_feature = int(in_feature)
        self.upsample = nn.Upsample(scale_factor=2,mode=UPSAMPLE_MODE)
        self.conv = nn.Conv1d(in_feature, int(in_feature/2),kernel_size=2, padding=PADDING_METHOD)
    
    def forward(self,x):
        out = self.upsample(x)
        out = self.conv(out)
        return out
    

class Unet1D(nn.Module):
    def __init__(self,in_feature):
        super(Unet1D,self).__init__()

        #LEFT PART
        self.maxpool = nn.MaxPool1d(2)

        self.contr1 = BottleneckResBlockContract(in_feature)
        #self.maxpool
        nb_f = in_feature*2
        self.contr2 = BottleneckResBlockContract(nb_f)
        #self.maxpool
        nb_f = nb_f*2
        self.contr3 = BottleneckResBlockContract(nb_f)
        #self.maxpool
        nb_f = nb_f*2
        #self.contr4 = BottleneckResBlockContract(nb_f)
        #self.maxpool
        #nb_f = nb_f*2
        ##############################################

        #DOWN PART
        
        self.contr5 = BottleneckResBlockContract(nb_f)
        nb_f = nb_f*2
        ##############################################

        #RIGHT PART

        #self.upsample4 = UpsamplingBlock(nb_f)
        #nb_f = nb_f/2
        #concat
        #nb_f = nb_f*2
        #self.expand4 = BottleneckResBlockExpand(nb_f)
        #nb_f = nb_f/2

        self.upsample3 = UpsamplingBlock(nb_f)
        nb_f = nb_f/2
        #concat
        nb_f = nb_f*2
        self.expand3 = BottleneckResBlockExpand(nb_f)
        nb_f = nb_f/2

        self.upsample2 = UpsamplingBlock(nb_f)
        nb_f = nb_f/2
        #concat
        nb_f = nb_f*2
        self.expand2 = BottleneckResBlockExpand(nb_f)
        nb_f = nb_f/2

        self.upsample1 = UpsamplingBlock(nb_f)
        nb_f = nb_f/2
        #concat
        nb_f = nb_f*2
        self.expand1 = BottleneckResBlockExpand(nb_f)
        nb_f = nb_f/2

        self.upsample0 = UpsamplingBlock(nb_f)
        nb_f = nb_f/2
        #concat
        nb_f = nb_f*2
        self.expand0 = BottleneckResBlockExpand(nb_f)
        nb_f = nb_f/2

    def forward(self,x):
        # X is of shape [nbatch,64,224]

        # LEFT PART
        l0 = x #[P,T]           #Output shape
        a = self.maxpool(l0)    #[P,T/2]
        l1 = self.contr1(a)     #[P*2,T/2]
        b = self.maxpool(l1)    #[P*2,T/4]
        l2 = self.contr2(b)     #[P*4,T/4]
        c = self.maxpool(l2)    #[P*4,T/8]
        l3 = self.contr3(c)     #[P*8,T/8]
        d = self.maxpool(l3)    #[P*8,T/16]
        #l4 = self.contr4(d)     #[P*16,T/16]
        #e = self.maxpool(l4)    #[P*16, T/32]

        # DOWN PART
        l5 = self.contr5(d)     #[P*32,T/32]

        # RIGHT PART
        #r4 = self.upsample4(l5)     #[P*16,T/16]
        #t4 = torch.cat((l4,r4),1)   #[P*32,T/16]
        #t4 = self.expand4(t4)       #[P*16,T/16]
        r3 = self.upsample3(l5)     #[P*8,T/8]
        t3 = torch.cat((l3,r3),1)   #[P*16,T/8]
        t3 = self.expand3(t3)       #[P*8,T/8]
        r2 = self.upsample2(t3)     #[P*4,T/4]
        t2 = torch.cat((l2,r2),1)   #[P*8,T/4]
        t2 = self.expand2(t2)       #[P*4,T/4]
        r1 = self.upsample1(t2)     #[P*2,T/2]
        t1 = torch.cat((l1,r1),1)   #[P*4,T/2]
        t1 = self.expand1(t1)       #[P*2,T/2]
        r0 = self.upsample0(t1)     #[P,T]
        t0 = torch.concat((l0,r0),1)#[P*2,T]
        t0 = self.expand0(t0)       #[P,T]

        return t0 # P features, T points

In [12]:
class PredictionHead(nn.Sequential):
    def __init__(self,n_features):
        super(PredictionHead,self).__init__()
        self.mlp = MLP2(n_features)
        self.unet = Unet1D(MLP_FILTERS_2)
        self.convHead = nn.Conv1d(MLP_FILTERS_2,N_CLASSES,kernel_size=3,padding=PADDING_METHOD)


In [13]:
class TemporalHead(nn.Sequential):
    def __init__(self,n_features):
        super(TemporalHead,self).__init__()   
        self.mlp = MLP2(n_features)
        self.unet = Unet1D(MLP_FILTERS_2)
        self.convHead = nn.Conv1d(MLP_FILTERS_2,N_CLASSES,kernel_size=3,padding=PADDING_METHOD)


### test loss

In [14]:
import torch.nn as nn

In [15]:
def huberLoss(pred, gTruth, treshold=0.5):
    res = pred - gTruth
    
    abs_res = torch.abs(res)
    quadratic_loss = 0.5 * torch.square(res)
    linear_loss = treshold * (abs_res - 0.5 * treshold)
    
    loss = torch.where(abs_res <= treshold, quadratic_loss, linear_loss)
    return torch.sum(loss)

In [16]:
def confLoss2(c_preds, c_truths):
    criterion = nn.BCEWithLogitsLoss(reduction='none')
    
    c_preds = c_preds.reshape(-1)
    c_truths = c_truths.reshape(-1)
    
    loss = criterion(c_preds, c_truths)
    total_loss = loss.sum()
    
    return total_loss

def dispLoss2(d_preds, d_truths, dom):
    #optimized version 
    
    dom = dom.bool()   # convert the domain mask to boolean tensor
    dp = torch.masked_select(d_preds, dom)   # select the predicted values where the mask is True
    dt = torch.masked_select(d_truths, dom)  # select the ground truth values where the mask is True
    loss = huberLoss(dp, dt)
    return loss

def valLoss(model_conf, model_disp, feat1, feat2, json_sols, dir_results, evaluator):
    model_conf.eval()
    model_disp.eval()
    
    av_map = evaluator.evaluate_map([model_conf, model_disp], feat1, feat2, SEQUENCE_SIZE, json_sols, dir_results)
    model_conf.train()
    model_disp.eval()
    return av_map["a_mAP"]
    

### Data loader

In [17]:
nbPerBatch = 20
nbPerSequence = 10
nbcL = len(cL)


In [18]:
dataLearning = BaiduDataset(data_l)
loaderLearning = data.DataLoader(dataLearning, batch_size = nbPerBatch, shuffle= True, num_workers = 2)

### Confidence

In [19]:
device = 'cuda'
model = PredictionHead(N_FEATURE_EXTRACTED)

In [20]:
wandb.init(project = 'dsProject', name = "window_64")
wandb.watch(model, confLoss2, log="all", log_freq=10)
#torch.onnx.export(model, images, "model_confidence.onnx")
#wandb.save("model_confidence.onnx")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mdelabrassinne (maxdelabrassinne). Use `wandb login --relogin` to force relogin


[]

In [35]:
classes = ['Kick-off', 
           'Ball out of play', 
           'Throw-in', 'Foul', 
           'Clearance', 
           'Indirect free-kick', 
           'Shots on target', 
           'Corner', 
           'Shots off target', 
           'Direct free-kick', 
           'Yellow card', 
           'Substitution', 
           'Red card', 
           'Offside', 
           'Goal', 
           'Penalty', 
           'Yellow->red card']

nms_windows = [20 for i in classes]

dir_results = "/scratch/users/mdelabrassinne/Results"

dir_model = "/home/mdelabrassinne/DSproject/yahoo/Reproducing"
conf_mod_path = os.path.join(dir_model, "confidence_head_test0.pt")
disp_mod_path = os.path.join(dir_model, "disp_head_test_base0.pt")

models = Network(conf_mod_path, disp_mod_path)

if not os.path.exists(dir_results):
    os.mkdir(dir_results)

evaluator = Evaluator(fps=FPS, nms_window = nms_windows, classes=classes)
    


In [ ]:
## Define loss
#loss_confidence = confLoss2

## Define optimizer
base_optimizer = torch.optim.Adam
optimizer = SAM(model.parameters(), base_optimizer, lr=0.001)

## put model on GPU
model.to(device)

## Train the model
num_epochs = 1000
for epoch in range(num_epochs):
    for features, coef_conf, coef_disp, dom, _ in tqdm(loaderLearning):
        
        features = features.view(-1, SEQUENCE_SIZE, 8576)
        coef_conf = coef_conf.view(-1, SEQUENCE_SIZE, nbcL)
        coef_disp = coef_disp.view(-1, SEQUENCE_SIZE, nbcL)
        dom = dom.view(-1, SEQUENCE_SIZE, nbcL)
        
        
        features, coef_conf, coef_disp, dom  = features.to(device), coef_conf.to(device), coef_disp.to(device), dom.to(device)  #check si obligé de tout mettre

        # Transpose for the way the NN is built
        dom = dom.transpose(1,2)
        coef_conf = coef_conf.transpose(1,2)
        coef_disp = coef_disp.transpose(1,2)
        
        T = features.shape[1]
        features = features.transpose(1, 2)
        
        # Forward pass
        outputs = model(features)
        
        # Apply the loss
        loss = confLoss2(outputs, coef_conf)
        
        wandb.log({"epoch": epoch, "loss_confidence": loss})
        
        
        # Backward and optimize
        
        # first step
        #optimizer.zero_grad()
        loss.backward()
        optimizer.first_step(zero_grad=True)
        #optimizer.step(closure)
        
        # second step
        
        confLoss2(model(features), coef_conf).backward()
        optimizer.second_step(zero_grad=True)
        
        #disp_mod = models.get_displacement_model()
        #av_map = valLoss(model, disp_mod, features1, features2, json_sols, dir_results, evaluator)
        #print("av_map", av_map)
        #wandb.log({"epoch": epoch, "av_map": av_map})

        #val = val.to("cpu")
        #images = images.to("cpu")
        
    # Print training loss after every epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    #disp_mod = models.get_displacement_model()
    #av_map = valLoss(model, disp_mod, features1, features2, json_sols, dir_results, evaluator)
    #wandb.log({"epoch": epoch, "av_map": av_map})
    
    if((epoch+1)%10 == 0):
        torch.save(model.state_dict(), "confidence_window_64.pt")

  0%|          | 0/45 [00:00<?, ?it/s]

torch.Size([200, 8576, 64])


/home/mdelabrassinne/anaconda3/envs/envDeepLearning/lib/python3.9/site-packages/torch/nn/modules/conv.py:309: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /opt/conda/conda-bld/pytorch_1670525551200/work/aten/src/ATen/native/Convolution.cpp:895.)
  return F.conv1d(input, weight, bias, self.stride,
  2%|▏         | 1/45 [01:26<1:03:07, 86.09s/it]

test


  4%|▍         | 2/45 [01:27<26:03, 36.35s/it]  

...

test
Epoch [13/1000], Loss: 5473.3827


  2%|▏         | 1/45 [01:21<59:59, 81.80s/it]

torch.Size([200, 8576, 64])
test


  4%|▍         | 2/45 [01:22<24:18, 33.93s/it]

torch.Size([200, 8576, 64])
test


### Displacement

In [22]:
#torch.save(model.state_dict(), "confidence_head_test1.pt")

In [23]:
model = TemporalHead(N_FEATURE_EXTRACTED)

wandb.init(project = 'dsProject_disp', name = "window_64")
wandb.watch(model, dispLoss2, log="all", log_freq=10)
#torch.onnx.export(model, images, "model_confidence.onnx")
#wandb.save("model_confidence.onnx")

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
loss_confidence,█▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁
epoch,20
loss_confidence,4310.02382


[]

In [ ]:

## Define loss
#loss_confidence = confLoss2

## Define optimizer
base_optimizer = torch.optim.Adam
optimizer = SAM(model.parameters(), base_optimizer, lr=0.001)

## put model on GPU
model.to(device)

## Train the model
num_epochs = 1000
for epoch in range(num_epochs):
    for features, coef_conf, coef_disp, dom, _ in tqdm(loaderLearning):
        features, coef_conf, coef_disp, dom  = features.to(device), coef_conf.to(device), coef_disp.to(device), dom.to(device)  #check si obligé de tout mettre
        
        features = features.view(-1, SEQUENCE_SIZE, 8576)
        coef_conf = coef_conf.view(-1, SEQUENCE_SIZE, nbcL)
        coef_disp = coef_disp.view(-1, SEQUENCE_SIZE, nbcL)
        dom = dom.view(-1, SEQUENCE_SIZE, nbcL)
        
        # Transpose for the way the NN is built
        dom = dom.transpose(1,2)
        coef_conf = coef_conf.transpose(1,2)
        coef_disp = coef_disp.transpose(1,2)
        
        T = features.shape[1]
        features = features.transpose(1, 2)
        
        
        # Forward pass
        outputs = model(features)
        
        # Compute the loss
        loss = dispLoss2(outputs, coef_disp, dom)
        
        wandb.log({"epoch": epoch, "loss_disp": loss})
        
        
        # Backward and optimize
        
        
         # first step
        #optimizer.zero_grad()
        loss.backward()
        optimizer.first_step(zero_grad=True)
        #optimizer.step(closure)
        
        # second step
        
        dispLoss2(model(features), coef_disp, dom).backward()
        optimizer.second_step(zero_grad=True)

    # Print training loss after every epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    
    if((epoch+1)%5 == 0):
        torch.save(model.state_dict(), "disp_window_64.pt")
        

torch.save(model.state_dict(), "disp_window_64.pt")

 49%|████▉     | 22/45 [14:26<12:45, 33.28s/it] 

In [27]:
torch.save(model.state_dict(), "disp_head_test0.pt")